In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from zipfile import ZipFile #압축파일 다루기위해
import os

In [2]:
df = pd.read_csv('train_weather.csv',encoding='cp949')
df = df.fillna(0)
df

,지점,지점명,일시,평균기온(°C),일강수량(mm),평균 풍속(m/s),평균 상대습도(%),가조시간(hr),합계 일조시간(hr),합계 일사량(MJ/m2),평균 지면온도(°C)
0,108,서울,2015-01-01,-7.7,0.0,4.6,41.4,9.6,8.9,9.79,-4.1
1,108,서울,2015-01-02,-6.0,0.0,3.2,45.9,9.6,7.6,9.07,-3.6
2,108,서울,2015-01-03,-2.7,0.0,1.9,56.1,9.7,7.9,8.66,-2.6
3,108,서울,2015-01-04,2.5,0.0,2.0,70.1,9.7,3.6,5.32,1.1
4,108,서울,2015-01-05,3.7,0.4,2.4,73.1,9.7,4.4,6.48,1.1
...,...,...,...,...,...,...,...,...,...,...,...
2552,108,서울,2021-12-27,-7.6,0.0,1.7,60.9,9.6,3.8,6.09,-4.7
2553,108,서울,2021-12-28,-4.1,0.0,2.2,73.8,9.6,1.7,4.32,-3.3
2554,108,서울,2021-12-29,0.4,0.2,2.6,72.9,9.6,1.8,4.64,-0.7
2555,108,서울,2021-12-30,-3.9,0.0,3.3,48.5,9.6,7.3,9.14,-2.7


In [3]:
df.columns

Index(['지점', '지점명', '일시', '평균기온(°C)', '일강수량(mm)', '평균 풍속(m/s)', '평균 상대습도(%)',
       '가조시간(hr)', '합계 일조시간(hr)', '합계 일사량(MJ/m2)', '평균 지면온도(°C)'],
      dtype='object')

In [4]:
weather_data = df.drop(['지점', '지점명', '일시'],axis=1)
train_data=weather_data[:2191]
train_mean,train_std =np.mean(train_data),np.std(train_data)
train_data=(train_data-train_mean)/train_std
train_data

c:\Users\wooji\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3438: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)


,평균기온(°C),일강수량(mm),평균 풍속(m/s),평균 상대습도(%),가조시간(hr),합계 일조시간(hr),합계 일사량(MJ/m2),평균 지면온도(°C)
0,-1.990799,-0.267965,3.129600,-1.170194,-1.456507,0.506461,-0.494644,-1.673009
1,-1.830178,-0.267965,1.305214,-0.871744,-1.456507,0.181188,-0.599625,-1.629542
2,-1.518384,-0.267965,-0.388858,-0.195256,-1.400541,0.256251,-0.659406,-1.542609
3,-1.027071,-0.267965,-0.258545,0.733257,-1.400541,-0.819651,-1.146401,-1.220956
4,-0.913692,-0.233648,0.262708,0.932224,-1.400541,-0.619484,-0.977265,-1.220956
...,...,...,...,...,...,...,...,...
2186,-1.055416,-0.267965,-0.519171,0.448071,-1.456507,0.331314,-0.611290,-1.368743
2187,-0.715277,-0.267965,-0.519171,0.255736,-1.456507,-0.244169,-0.738142,-1.194876
2188,-0.630242,-0.156437,-1.040424,0.865902,-1.456507,0.281272,-0.592335,-1.081863
2189,-1.253831,-0.250807,0.914274,0.779683,-1.456507,-1.720407,-1.633396,-1.325276


In [5]:
sequencelength=365 #학습 데이터를 200개 단위로 잘라서 학습
predict_step=365 #10시간 후가 60개 데이터 포인트이므로, 라벨 데이터로 x1 데이터의 60 step 이후의 데이터를 y1데이터로 설정
batch_size=1024 #batch size 설정

datax=train_data[:-(sequencelength+predict_step)]
datay=train_data[sequencelength+predict_step:] #기온을 예측하기 때문에 기온 정보만을 라벨 데이터로 설정
train_dataset = keras.preprocessing.timeseries_dataset_from_array(datax, datay, sequence_length=sequencelength, batch_size=batch_size)

In [6]:
feature_number=8 #feature:T (degC), p (mbar), VPmax (mbar), sh (g/kg), wv (m/s)
inputs = keras.Input(shape=(sequencelength,feature_number))
x = layers.LSTM(units=100)(inputs) #LSTM layer로 학습 모델을 구성
outputs = keras.layers.Dense(8)(x)
model = keras.Model(inputs=inputs, outputs=outputs)
model.compile(optimizer=Adam(learning_rate=0.001), loss='mse', metrics=["mean_absolute_error"])

# loss 및 metric으로 mse(평균제곱오차) 설정
model.fit(train_dataset,epochs=30)

Epoch 1/30
2/2 [==============================] - 4s 163ms/step - loss: 1.1092 - mean_absolute_error: 0.8383
Epoch 2/30
2/2 [==============================] - 1s 166ms/step - loss: 0.9720 - mean_absolute_error: 0.7778
Epoch 3/30
2/2 [==============================] - 1s 166ms/step - loss: 0.8652 - mean_absolute_error: 0.7235
Epoch 4/30
2/2 [==============================] - 1s 164ms/step - loss: 0.7758 - mean_absolute_error: 0.6705
Epoch 5/30
2/2 [==============================] - 1s 161ms/step - loss: 0.7032 - mean_absolute_error: 0.6193
Epoch 6/30
2/2 [==============================] - 1s 164ms/step - loss: 0.6525 - mean_absolute_error: 0.5790
Epoch 7/30
2/2 [==============================] - 1s 161ms/step - loss: 0.6272 - mean_absolute_error: 0.5610
Epoch 8/30
2/2 [==============================] - 1s 166ms/step - loss: 0.6166 - mean_absolute_error: 0.5554
Epoch 9/30
2/2 [==============================] - 1s 165ms/step - loss: 0.6097 - mean_absolute_error: 0.5509
Epoch 10/30
2/2 [==

In [7]:
metrics=model.evaluate(train_dataset, verbose=2)
print("Train Data MAE:", metrics[1])

2/2 - 2s - loss: 0.5545 - mean_absolute_error: 0.4936 - 2s/epoch - 751ms/step
Train Data MAE: 0.4935552477836609


In [8]:
test_data=weather_data[2191:]
test_mean,test_std =np.mean(test_data),np.std(test_data)
test_data=(test_data-test_mean)/test_std

c:\Users\wooji\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3438: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)


In [9]:
test_datax=test_data[:-(sequencelength+predict_step)]
test_datay=test_data[sequencelength+predict_step:]
test_dataset = keras.preprocessing.timeseries_dataset_from_array(test_datax, test_datay, sequence_length=sequencelength, batch_size=batch_size)

ValueError: `sampling_rate` must be lower than the length of the data. Received: sampling_rate=1, for data of length 0